# 1. Exploring 

In [1]:
from bs4 import BeautifulSoup
from urllib.request import urlopen
from selenium import webdriver
#Define the webdriver 
from selenium.webdriver import Chrome
from selenium.webdriver.chrome.options import Options

import requests #to retrieve HTML and transfer
import time


In [2]:
opts = Options()
opts.set_headless()
assert opts.headless # Operating in headless mode

#Insert link for scraping
browser = Chrome(options=opts)
browser.get('https://forum.flowster.app/')

/Users/mac/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: DeprecationWarning: use setter for headless property instead of set_headless
  


# 2. Get Category and Topic

In [3]:
category_table = browser.find_elements_by_class_name('category-list')
print(category_table[0].text)

Category Topics
Amazon Specific
Have questions about using the Amazon marketplace? This is the category to use. Please be sure to select the most appropriate sub-category for your questions.
All Other Amazon Related Issues
Software & Tools
PPC
Reviews & Customer Service
Inventory Management
Pricing Management
Product Prep
59
Flowster-specific
Discussion about Flowster, this Forum’s organization, and how we can improve it.
Updates
Free SOPs
Marketplace
61
Store & Website Management
Have questions about Store & Website Management? This is the category to use. Please be sure to select the most appropriate sub-category for your questions.
Product Detail Pages
Copywriting
Video & Photography
Branding
Payments & Fraud
2
Product Sourcing
Have questions about sourcing products? This is the category to use. Please be sure to select the most appropriate sub-category for your questions.
Purchasing
Sourcing Brand Name Products
Vendor Management
Pricing & Negotiation
Vendor Relations
Product Creati

In [4]:
categories = browser.find_elements_by_class_name('category-text-title')
categories[1].text

'Flowster-specific'

In [5]:
categ_links = browser.find_elements_by_css_selector('.category > h3 > a')
categ_links[2].get_attribute('href')
###############

'https://forum.flowster.app/c/store-website-management/40'

In [6]:
#Number of catergories
len(categ_links)

12

In [7]:
categ_names = []
categ_urls = []
for categ, link in zip(categories, categ_links):
    categ_names.append(categ.text)
    categ_urls.append(link.get_attribute('href'))

In [8]:
categ_names, categ_urls

(['Amazon Specific',
  'Flowster-specific',
  'Store & Website Management',
  'Product Sourcing',
  'Management',
  'Fulfillment',
  'eCommerce Marketplaces',
  'Traffic Sources',
  'Software & Tools',
  'Financial Management',
  'Human Resources',
  'Misc Topics'],
 ['https://forum.flowster.app/c/amazon-specific/17',
  'https://forum.flowster.app/c/flowster/2',
  'https://forum.flowster.app/c/store-website-management/40',
  'https://forum.flowster.app/c/product-sourcing/8',
  'https://forum.flowster.app/c/management/32',
  'https://forum.flowster.app/c/fulfillment/36',
  'https://forum.flowster.app/c/sales-channels/7',
  'https://forum.flowster.app/c/traffic/13',
  'https://forum.flowster.app/c/software-tools/26',
  'https://forum.flowster.app/c/financial-management/21',
  'https://forum.flowster.app/c/human-resources/46',
  'https://forum.flowster.app/c/misc-topics/55'])

In [9]:
#Access category by using url


CATEG_URL = categ_links[2].get_attribute('href')
#page = requests.get(SEC_URL)
browser.get(CATEG_URL)
browser.execute_script("window.scrollTo(0,document.body.scrollHeight)")
time.sleep(3)
#print(browser.page_source)

In [10]:
# Get page content
soup = BeautifulSoup(browser.page_source, 'html.parser')

In [11]:
storeWebM_topics_links = soup.find_all('a', class_='title raw-link raw-topic-link')
len(storeWebM_topics_links)

3

In [12]:
storeWebM_topics_links[0]['href']

'/t/about-the-store-website-management-category/58'

In [13]:
storeWebM_topics_urls =[]

BASE_URL = 'https://forum.flowster.app'
for topic_link in storeWebM_topics_links:
    storeWebM_topics_urls.append(BASE_URL + topic_link['href'])
    
storeWebM_topics_urls

['https://forum.flowster.app/t/about-the-store-website-management-category/58',
 'https://forum.flowster.app/t/securing-long-term-partnerships/1545',
 'https://forum.flowster.app/t/amazon-free-products/1470']

In [14]:
topic_title = soup.find_all('a', class_='title raw-link raw-topic-link')

In [15]:
topic_titles = []

for title in topic_title:
    topic_titles.append(title.text)
    
topic_titles

['About the Store & Website Management category',
 'Securing long term partnerships',
 'Amazon Free Products']

# 3. Get topic post, like

In [16]:
secondtopic_url = storeWebM_topics_urls[1]
#page = requests.get(SEC_URL)
browser.get(secondtopic_url)
browser.execute_script("window.scrollTo(0,document.body.scrollHeight)")
time.sleep(3)

In [17]:
secondtopic_soup = BeautifulSoup(browser.page_source, 'html.parser')

In [18]:
#Get all posts
postStream = secondtopic_soup.find('div', class_='post-stream')

In [19]:
#Get post and post's replies then extract
postDivs = postStream.find_all('div', {'class': ['topic-post clearfix topic-owner regular', 'topic-post clearfix regular']})
#????????#

comments = []
for i in range(len(postDivs)):
    comment = postDivs[i].find('div', class_='cooked').text
    comments.append(comment)
    
leading_comment = comments[0]

if len(comments) == 1:
    other_comments = ('no other comment')
else:
    other_comments = comments[1:]

In [20]:
leading_comment

'Hello, I just closed a deal to manage a quite large brand account on Amazon, my main concern now is that this partnership may only last 1-2 years and once the account has grown to higher levels they would just manage it by themselves. What methods, tips and tricks you can suggest to protect this deal and make sure they will stick with my company managing their account for long term? Essentially how can I make them dependent on me so it will be very hard for them to let go our partnership?'

In [21]:
other_comments

['Brand management is going to have partner turnover.  There will always be high expectations and, even if you raise sales 20%, they’ll want a 30% increase the next quarter.  I think one good tip is to set reasonable expectations.  Let them know all the activities you will undertake and don’t over-promise a sales increase.']

In [22]:
created = postStream.find('li', class_= "created-at")
created_at = created.find('span', class_= 'relative-date')
created_at['title']

'Dec 5, 2020 5:49 pm'

In [23]:
last_reply = postStream.find('li', class_= "last-reply")
last_reply_at = last_reply.find('span', class_= 'relative-date')['title']
last_reply_at

'Dec 7, 2020 3:02 pm'

In [24]:
replies = postStream.find('li', class_="replies")
nbr_replies = replies.find('span', class_='number').text
nbr_replies

'1'

In [25]:
views = postStream.find('li', class_="secondary views")
nbr_views = views.find('span', class_='number').text
nbr_views

'115'

# 4. Scraper Class

In [26]:
from datetime import datetime
import os
from os.path import expanduser

import pandas as pd
import json

In [48]:
class WebScraper:
    browser = None
    topic_dict = {}
    topic_df = pd.DataFrame(columns = [
        'topic title',
        'category',
        'tags',
        'leading post', 
        'post replies',
        'created_at', 
        'like',
        'view',
        'replies',
    ])
    def __init__(self, webdriverpath): #2 underscores
        opts = Options()
        opts.set_headless()
        assert opts.headless
        self.browser = Chrome(options=opts, executable_path=webdriverpath)
        
    def get_topic(self, topic_soup):
        
        topic_titles = topic_soup.find_all('a', class_='title raw-link raw-topic-link')
        topic_title = []
        for title in topic_titles:
            topic_title.append(title.text)
            
        
        topic_wrapper = topic_soup.find('div', class_ = 'title-wrapper')
        topic_tag = topic_wrapper.find_all('span', class_='category-name')
        topic_tag = [tag.text for tag in topic_tag]
        try:
            topic_category = topic_tag[0]
            if len(topic_tag)==1:
                topic_tag = ('no more topic tag')
            else:
                topic_tag = topics_tag[1:]
        except: #????????#
            topic_category = ''
            topic_tag = ''
        
        return topic_title, topic_category, topic_tag
    
    
    def get_topic_comment(self, topic_soup):
        postStream = topic_soup.find('div', class_='post-stream')
        postsDivs = postStream.find_all('div', {'class': ['topic-post clearfix topic-owner regular', 'topic-post clearfix regular']})

        comments = []
        for i in range(len(postsDivs)):
            comment = postsDivs[i].find('div', class_='cooked').text
            comments.append(comment)
        try:
            leading_comment = comments[0]
            if len(comments) == 1:
                other_comments = ('no more comment')
            else:
                other_comments = comments[1:]
        except:
            leading_comment, other_comments = [], []
        
        return leading_comment, other_comments
    
    
    def get_topic_created_at(self, topic_soup):
        created = topic_soup.find('li', class_="created-at")
        
        if created is None:
            created_at = str(0)
        else:
            created_at = created.find('span', class_='relative-date')['title']
    
        return created_at
    
    
    def get_topic_replies_nbr(self, topic_soup):
        replies = topic_soup.find('li', class_="replies")
        
        if replies == None:
            nbr_replies = str(0)
        else:
            nbr_replies = replies.find('span', class_='number').text
        
        return nbr_replies

    
    def get_topic_views_nbr(self, topic_soup):
        views = topic_soup.find('li', class_="secondary views")
        
        if views is None:
            nbr_views = str(0)
        else:
            nbr_views = views.find('span', class_='number').text
        
        return nbr_views

    
    def get_topic_likes_nbr(self, topic_soup):
        likes = topic_soup.find('li', class_="secondary likes")
        
        if likes is None:
            nbr_likes = str(0)
        else:
            nbr_likes = likes.find('span', class_='number').text
        
        return nbr_likes
    
    
    def Scraper(self, base_url, site_name):
        
        #1. Retrieve page source in browser
        self.browser.get(base_url)
        
        #2. Get all category link
        categ_links = self.browser.find_elements_by_css_selector('.category > h3 > a')
        categ_urls = []
        for link in categ_links:
            categ_urls.append(link.get_attribute('href'))
        
        for categ_url in categ_urls:
            self.browser.get(categ_url)
        
            #??????????#
            # Load the entire webage by scrolling to the bottom
            lastHeight = self.browser.execute_script("window.scrollTo(0,document.body.scrollHeight)")
            
            while (True):
                # Scroll to bottom of page
                self.browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")

                # Wait for new page segment to load
                time.sleep(0.5)

                # Calculate new scroll height and compare with last scroll height
                newHeight = self.browser.execute_script("return document.body.scrollHeight")
                if newHeight == lastHeight:
                    break
                    
                lastHeight = newHeight
            #4. Create category soup
            categoryHTML = self.browser.page_source
            categ_topic_soup = BeautifulSoup(categoryHTML, 'html.parser')
    
            categ_topic_links = categ_topic_soup.find_all('a', class_='title raw-link raw-topic-link')
    
            # Get all the topic urls 
            categ_topic_urls = []
            for topic_link in categ_topic_links:
                categ_topic_urls.append(base_url + topic_link['href'])
                
            for categ_topic_url in categ_topic_urls:
                # Get current topic_soup
                self.browser.get(categ_topic_url)
                topicHTML = self.browser.page_source
                topic_soup = BeautifulSoup(topicHTML, 'html.parser')
                
                # Scrape all topic attributes of interest
                topic_title, topic_category, topic_tag = self.get_topic(topic_soup)
                leading_comment, other_comments = self.get_topic_comment(topic_soup)
                created_at = self.get_topic_created_at(topic_soup)
                nbr_replies = self.get_topic_replies_nbr(topic_soup)
                nbr_views = self.get_topic_views_nbr(topic_soup)
                nbr_likes = self.get_topic_likes_nbr(topic_soup)
                
                #Add to dictionary
                attribute_dict = {
                    'topic title': topic_title,
                    'category': topic_category,
                    'tags': topic_tag,
                    'leading post': leading_comment, 
                    'post replies': other_comments,
                    'created_at': created_at, 
                    'like': nbr_likes,
                    'view': nbr_views,
                    'replies': nbr_replies}
                
                #self.topic_dict[topic_title] = attribute_dict
                self.topic_dict = attribute_dict
                self.topic_df = self.topic_df.append(attribute_dict, ignore_index = True)
                
                #test
                #print('Title :', topic_title)
                #print('Category :', topic_category)
                #print('URL :', categ_topic_url)
                
        #timestamp
        timeStamp = datetime.now().strftime('%Y%m%d%H%M%S')
        
        #4. Save data in json and csv
        #jsonFilename = site_name + '_SCRAPED_DATA_' + timeStamp + '.json'
        csvFilename = site_name + '_SCRAPED_DATA_' + timeStamp + '.csv'
        
        #jsonFileFullPath = os.path.join(os.path.dirname(os.path.realpath(__file__)), jsonFilename)
        csvFileFullPath = os.path.join(os.path.dirname(os.path.realpath(__file__)), csvFilename)
        
        #userhome = expanduser("~")
        #csvFileFullPath = os.path.join(userhome, 'Desktop', csvFilename)
        #open(csvFileFullPath, "r")
        
        # Save scraped data  into json file
        #with open(jsonFileFullPath, 'w') as f:
            #json.dump(self.topic_dict, f)
        
        # Save dataframe into csv file
        self.topic_df.to_csv(csvFileFullPath)
        
        #final = self.topic_df
        
        #return(final)
        #print(final)

In [49]:
if __name__=='__main__':
    # Local path to webdriver 
    webdriverpath = '/usr/local/bin/chromedriver'
    ##terminal: which chromedriver
    ##rm restrict: xattr -d com.apple.quarantine /usr/local/bin/chromedriver
    
    
    # Forum to scrape URL    
    base_url= 'https://forum.flowster.app'
    
    # Name of the forum
    site_name = 'FLOWSTER'
        
    # WebScraping object
    webScraper = WebScraper(webdriverpath)
    
    # Run the webscraper and save scraped data
    webScraper.Scraper(base_url, site_name)

/Users/mac/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:17: DeprecationWarning: use setter for headless property instead of set_headless


NameError: name '__file__' is not defined

In [38]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re #for regular expressions
import string #for handling string
import math # for mathematical operations

In [42]:

url = 'https://raw.githubusercontent.com/mentorchains/level1_post_recommender_20/md2/webScraping_EDA_tutorials/FLOWSTER_SCRAPED_DATA20201212224954.csv'
data = pd.read_csv(url,index_col=0)
print(data.shape)

(278, 9)


In [40]:
data.head(5)

,Topic Title,Category,Tags,Leading Post,Post Replies,Created_at,Likes,Views,Replies
0,About the Store & Website Management category,Store & Website Management,[],Have questions about Store & Website Managemen...,[],0,0,0,0
1,Securing long term partnerships,Store & Website Management,[],"Hello, I just closed a deal to manage a quite ...",['Brand management is going to have partner tu...,"Dec 5, 2020 5:49 pm",0,19,1
2,Amazon Free Products,Store & Website Management,[],"Hello,\nI need some buyers for my products in ...",[],0,0,0,0
3,About the Product Sourcing Category,Product Sourcing,[],Have questions about sourcing products? This i...,[],0,0,0,0
4,Virtual Assistant sending Emails Hubspot,Product Sourcing,[],"Hello Bright Ideas Tribe,\nI would like to giv...",['\n\n\nknowledge.hubspot.com 1\n\n\n\nCreate ...,"Nov 18, 2020 9:12 pm",0,81,5


In [43]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 278 entries, 0 to 277
Data columns (total 9 columns):
Topic Title     278 non-null object
Category        278 non-null object
Tags            278 non-null object
Leading Post    277 non-null object
Post Replies    278 non-null object
Created_at      278 non-null object
Likes           278 non-null int64
Views           278 non-null int64
Replies         278 non-null int64
dtypes: int64(3), object(6)
memory usage: 21.7+ KB


In [45]:
data['Leading Post'][1]

'Hello, I just closed a deal to manage a quite large brand account on Amazon, my main concern now is that this partnership may only last 1-2 years and once the account has grown to higher levels they would just manage it by themselves. What methods, tips and tricks you can suggest to protect this deal and make sure they will stick with my company managing their account for long term? Essentially how can I make them dependent on me so it will be very hard for them to let go our partnership?'